In [1]:
from pathlib import Path
import pandas as pd

from make_summary import summarize_group
from mil.utils import human_format

selected_model_type = "mnist_collage"
selected_model_type = "mnist_collage_ablations"
yaml_folder = Path("conf") / "selected_model" / selected_model_type

/cs/home/gw66/das-mil/mil_env/lib64/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = [file.stem for file in yaml_folder.glob("*.yaml") if file.name[0] != "_"]
if selected_model_type == "mnist_collage_ablations":
    models.append("distance_aware_self_attention")
models.sort()
models

['distance_aware_self_attention',
 'distance_aware_self_attention_embedk',
 'distance_aware_self_attention_embedkq',
 'distance_aware_self_attention_embedkq_noterm3',
 'distance_aware_self_attention_embedkqv_noterm3',
 'distance_aware_self_attention_embedkv',
 'distance_aware_self_attention_embedq',
 'distance_aware_self_attention_embedqv',
 'distance_aware_self_attention_embedv',
 'distance_aware_self_attention_fixedembed']

In [3]:
mnist_collage_stats = {
    model: summarize_group(f"selected-mnist_collage-{model}", log_to_wandb=False)
    for model in models
}
mnist_collage_inverse_stats = {
    model: summarize_group(f"selected-mnist_collage_inverse-{model}", log_to_wandb=False)
    for model in models
}
df_mnist_collage = pd.DataFrame(mnist_collage_stats).T
df_mnist_collage_inverse = pd.DataFrame(mnist_collage_inverse_stats).T

# Merge
df_mnist_collage["dataset"] = "mnist_collage"
df_mnist_collage_inverse["dataset"] = "mnist_collage_inverse"
df = pd.concat([df_mnist_collage, df_mnist_collage_inverse])
df.index.name = "model"
df.reset_index(inplace=True)
df.set_index(["dataset", "model"], inplace=True)
df

2023-05-07 12:53:57.111 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention
2023-05-07 12:53:59.287 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention_embedk
2023-05-07 12:54:00.841 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention_embedkq
2023-05-07 12:54:03.034 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention_embedkq_noterm3
2023-05-07 12:54:04.705 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention_embedkqv_noterm3
2023-05-07 12:54:07.361 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention_embedkv
2023-05-07 12:54:09.908 | INFO     | make_summary:summarize_group:43 - Summarizing group sel

mean(train/acc)  \
dataset               model                                                             
mnist_collage         distance_aware_self_attention                          0.932000   
                      distance_aware_self_attention_embedk                   0.928667   
                      distance_aware_self_attention_embedkq                  0.899333   
                      distance_aware_self_attention_embedkq_noterm3          0.918000   
                      distance_aware_self_attention_embedkqv_noterm3         0.954667   
                      distance_aware_self_attention_embedkv                  0.940000   
                      distance_aware_self_attention_embedq                   0.918667   
                      distance_aware_self_attention_embedqv                  0.926667   
                      distance_aware_self_attention_embedv                   0.926000   
                      distance_aware_self_attention_fixedembed               0.936667   
mnist_collage_inverse distance_aware_self_attention                          0.910000   
                      distance_aware_self_attention_embedk                   0.870000   
                      distance_aware_self_attention_embedkq                  0.906667   
                      distance_aware_self_attention_embedkq_noterm3          0.926667   
                      distance_aware_self_attention_embedkqv_noterm3         0.952667   
                      distance_aware_self_attention_embedkv                  0.948000   
                      distance_aware_self_attention_embedq                   0.848000   
                      distance_aware_self_attention_embedqv                  0.954000   
                      distance_aware_self_attention_embedv                   0.917333   
                      distance_aware_self_attention_fixedembed               0.932000   

                                                                      std(train/acc)  \
dataset               model                                                            
mnist_collage         distance_aware_self_attention                         0.028829   
                      distance_aware_self_attention_embedk                  0.058907   
                      distance_aware_self_attention_embedkq                 0.050848   
                      distance_aware_self_attention_embedkq_noterm3         0.034849   
                      distance_aware_self_attention_embedkqv_noterm3        0.010698   
                      distance_aware_self_attention_embedkv                 0.022973   
                      distance_aware_self_attention_embedq                  0.013864   
                      distance_aware_self_attention_embedqv                 0.015456   
                      distance_aware_self_attention_embedv                  0.016566   
                      distance_aware_self_attention_fixedembed              0.013123   
mnist_collage_inverse distance_aware_self_attention                         0.048362   
                      distance_aware_self_attention_embedk                  0.072839   
                      distance_aware_self_attention_embedkq                 0.042622   
                      distance_aware_self_attention_embedkq_noterm3         0.018708   
                      distance_aware_self_attention_embedkqv_noterm3        0.004346   
                      distance_aware_self_attention_embedkv                 0.008367   
                      distance_aware_self_attention_embedq                  0.108002   
                      distance_aware_self_attention_embedqv                 0.011879   
                      distance_aware_self_attention_embedv                  0.030129   
                      distance_aware_self_attention_fixedembed              0.045252   

                                                                      mean(train/balanced_acc)  \
dataset               model                                                 

In [5]:
if selected_model_type == "mnist_collage":
    NAMES = {
        "just_pool": "MIL with max pool",
        "abmil": "AB-MIL~\\cite{ilse2018attention}",
        "gnn_gat": "MIL with GNN (GAT~\\cite{velickovic2018graph})",
        "gnn_gcn": "MIL with GNN (GCN~\\cite{kipf2017semisupervised})",
        "induced_set_transformer": "MIL with iSet Transformer~\\cite{lee2019set}",
        "set_transformer": "MIL with Set Transformer~\\cite{lee2019set}",
        "mil_gnn": "MIL-GNN~\\cite{tu2019multiple}",
        "mil_gnn_ds": "MIL-GNN-DS~\\cite{tu2019multiple}",
        "self_attention": "MIL with SA~\\cite{vaswani2017attention}",
        "self_attention_axial_pe": "MIL with SA + axial PE~\\cite{ramachandran2019stand}",
        "self_attention_fourier_pe": "MIL with SA + Fourier PE~\\cite{yang2021learnable}",
        "discrete_rel_pos_self_attention": "MIL with disc.\\ rel.\\ SA~\\cite{wu2021rethinking}",
        "transmil": "Trans-MIL~\\cite{shao2021transmil}",
        "distance_aware_self_attention": "DAS-MIL (ours)",
    }
elif selected_model_type == "mnist_collage_ablations":
    NAMES = {
        "distance_aware_self_attention_embedk": "DAS-MIL ($\\vb^K$)",
        "distance_aware_self_attention_embedq": "DAS-MIL ($\\vb^Q$)",
        "distance_aware_self_attention_embedv": "DAS-MIL ($\\vb^V$)",
        "distance_aware_self_attention_embedkq": "DAS-MIL ($\\vb^K, \\vb^Q$) (+ \\vb^Q{\\vb^K}^\\top in \\cref{eq:das:compatibility:impl})",
        "distance_aware_self_attention_embedkq_noterm3": "DAS-MIL ($\\vb^K, \\vb^Q$)",
        "distance_aware_self_attention_embedkv": "DAS-MIL ($\\vb^K, \\vb^V$)",
        "distance_aware_self_attention_embedqv": "DAS-MIL ($\\vb^Q, \\vb^V$)",
        "distance_aware_self_attention_embedkqv": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$) (+ \\vb^Q{\\vb^K}^\\top in \\cref{eq:das:compatibility:impl})",
        "distance_aware_self_attention_fixedembed": "DAS-MIL (non-trainable $\\vb^K, \\vb^Q, \\vb^V$)",
        "distance_aware_self_attention_embedkqv_noterm3": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$)",
        # "distance_aware_self_attention": "DAS-MIL ($\\vb^K, \\vb^Q, \\vb^V$) (+ \\vb^Q{\\vb^K}^\\top in \\cref{eq:das:compatibility:impl})",
    }

pos_col = selected_model_type == "mnist_collage"
def if_pos(s):
    return s if pos_col else ""

abs_pos = ["self_attention_axial_pe", "self_attention_fourier_pe", "transmil"]
rel_pos = ["discrete_rel_pos_self_attention", "distance_aware_self_attention", "mil_gnn", "mil_gnn_ds", "gnn_gat", "gnn_gcn"]

print("\\begin{tabular}{l" + if_pos("|c") + "|r|rr|rr}")
print("\\toprule")
print(" & " + if_pos("&") + " & \\multicolumn{2}{c|}{\\smaller{MNIST-COLLAGE}} & \\multicolumn{2}{c}{\\smaller{MNIST-COLLAGE-INV}} \\\\")
print("Model " + if_pos("& \\multicolumn{1}{c|}{Pos}") + " & \\multicolumn{1}{c|}{Params} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c|}{Test} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c}{Test} \\\\")
print("\\midrule")

rows = {name: row for name, row in df.iterrows()}
for name, desc in NAMES.items():
    pos = "abs" if name in abs_pos else "rel" if name in rel_pos else None
    row = f"{NAMES[name]}"
    pos = f" \\smaller{{{pos.upper()}}}" if pos in ("abs", "rel") else "\\xmark"
    row += if_pos(f" & {pos}")
    row += f" & {human_format(df.loc['mnist_collage', name]['num_parameters'])}"

    metric = "balanced_acc"

    for dataset in ("mnist_collage", "mnist_collage_inverse"):
        df_ds = df.loc[dataset]
        for split in ("train", "test"):
            if df_ds[f"mean({split}/{metric})"].max() == df_ds[f"mean({split}/{metric})"].loc[name]:
                row += f" & \\textbf{{{df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}}}"
            else:
                row += f" & {df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}"
    print(row + " \\\\")
print("\\bottomrule")
print("\\end{tabular}")

\begin{tabular}{l|r|rr|rr}
\toprule
 &  & \multicolumn{2}{c|}{\smaller{MNIST-COLLAGE}} & \multicolumn{2}{c}{\smaller{MNIST-COLLAGE-INV}} \\
Model  & \multicolumn{1}{c|}{Params} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c|}{Test} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c}{Test} \\
\midrule
DAS-MIL ($\vb^K$) & 17.3K & 0.929 $\pm$ 0.059 & 0.910 $\pm$ 0.029 & 0.870 $\pm$ 0.073 & 0.836 $\pm$ 0.077 \\
DAS-MIL ($\vb^Q$) & 17.3K & 0.919 $\pm$ 0.014 & 0.914 $\pm$ 0.055 & 0.848 $\pm$ 0.108 & 0.836 $\pm$ 0.121 \\
DAS-MIL ($\vb^V$) & 17.3K & 0.926 $\pm$ 0.017 & 0.912 $\pm$ 0.015 & 0.917 $\pm$ 0.030 & 0.890 $\pm$ 0.029 \\
DAS-MIL ($\vb^K, \vb^Q$) & 17.3K & 0.899 $\pm$ 0.051 & 0.906 $\pm$ 0.065 & 0.907 $\pm$ 0.043 & 0.850 $\pm$ 0.062 \\
DAS-MIL ($\vb^K, \vb^V$) & 17.3K & 0.940 $\pm$ 0.023 & 0.934 $\pm$ 0.019 & 0.948 $\pm$ 0.008 & \textbf{0.942 $\pm$ 0.028} \\
DAS-MIL ($\vb^Q, \vb^V$) & 17.3K & 0.927 $\pm$ 0.015 & 0.916 $\pm$ 0.013 & \textbf{0.954 $\pm$ 0.012} & 0.912 $\pm$ 0.026 \\
DAS-MIL